# imports

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F # relu, tanh # also included in nn package
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# create network

In [16]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes): # (28x28) = 784 nodes
        super(NN, self).__init__() # call init of nn.Module
        self.fc1 = nn.Linear(input_size, 50) # 50 hidden nodes
        self.fc2 = nn.Linear(50, num_classes)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # no nonlinearity
        return x
    
model = NN(784, 10)
x = torch.randn(64, 784)
model(x).shape

torch.Size([64, 10])

# hyperparameters

In [17]:
input_size = 784
num_classes = 10
lr = 0.001
batch_size = 64
num_epochs = 1

# load data

In [18]:
train_dataset = datasets.MNIST(root='dataset/', train=True,
                               transform=transforms.ToTensor(),
                               download=True) # training_set
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False,
                               transform=transforms.ToTensor(),
                               download=True) # training_set
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# initialize network

In [19]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

# loss and optimizer

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# train network

In [21]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # put data on device
        data = data.to(device)
        targets = targets.to(device)
        
        # make correct shape
        data = data.reshape(data.shape[0], -1) # 64 by the rest

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad() # set all gradients to 0 for each batch
                              # to not save gradient for each minibatch
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

# check accuracy

In [22]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    # don't need to compute gradients when checking accuracy
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device)
            y = y.to(device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, preds = scores.max(1) # index of maximum value for second dim
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 55707 / 60000 with accuracy 92.84
Checking accuracy on test data
Got 9302 / 10000 with accuracy 93.02
